In [326]:
# Librerias generales

# Pandas
import pandas as pd
import math
pd.set_option('display.max_columns', 50) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Número máximo de filas a mostar

# Random seed
import numpy as np
np.random.seed(3301)

# Seaborn
import seaborn as sns 

# Matplolib
%matplotlib inline
import matplotlib.pyplot as plt

# math


# scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

db_location = './Datos/datos_entrenamiento_laboratorio1(train_data).csv'
data = pd.read_csv(db_location, sep=',')
data.shape 

from joblib import dump, load


In [327]:
import pandas as pd
import numpy as np

def limpiar_datos(df: pd.DataFrame) -> pd.DataFrame:

    df = df.copy()

    # 1) market_value → precio_millones (en unidades monetarias, no “millones”)
    if 'market_value' in df.columns:
        mv = (
            df['market_value']
            .astype('string')
            .str.lower()
            .str.strip()
            .replace({'-': np.nan, 'error': np.nan})
        )
        # Captura número con posible decimal y sufijo m/k
        ext = mv.str.extract(r'€?\s*([\d]+(?:\.[\d]+)?)\s*([mk]?)', expand=True)
        ext.columns = ['cantidad', 'sufijo']
        cantidad = pd.to_numeric(ext['cantidad'], errors='coerce')  # respeta '.' como decimal
        mult = ext['sufijo'].map({'m': 1_000_000.0, 'k': 1_000.0}).fillna(1.0)
        df['precio_millones'] = cantidad * mult
    
        df = df[df['precio_millones'] < 1e10*0.1]

    # 2) Dia_partido: quitar sufijos y parsear con formato
    if 'Dia_partido' in df.columns:
        col = df['Dia_partido'].astype('string')
        for suf in [" North", " West", " Derby", " El", " Choc"]:
            col = col.str.replace(suf, "", regex=False)
        df['Dia_partido'] = pd.to_datetime(col, format="%A %B %d, %Y")

    # 3) Edad: normalizar y convertir a float
    if 'Edad' in df.columns:
        df['Edad'] = (
            df['Edad'].astype('string').str.replace('-', '.', regex=False)
        )
        df['Edad'] = df['Edad'].astype(float)  # sin errors='coerce' para replicar el lab

    # 4) Correcciones puntuales de Nacionalidad (si aplica)
    if {'Jugador', 'Nacionalidad'}.issubset(df.columns):
        df.loc[df['Jugador'].eq('Fer Lopez'), 'Nacionalidad'] = 'es ESP'
        df.loc[df['Jugador'].eq('Mateus Mane'), 'Nacionalidad'] = 'pt POR'
    
    # 5) Quitar columnas duplicadas (por contenido)
    df = df.loc[:, ~df.T.duplicated()]

    # 6) Goles entre 0 y 15
    if 'Goles' in df.columns:
        df = df[(df['Goles'] >= 0) & (df['Goles'] <= 15)]
    
    df = pd.get_dummies(df, columns=['Nacionalidad'], drop_first=False, dtype=float)

    number_columns = df.select_dtypes(include=[np.number]).columns
    return df[number_columns]






In [328]:
# datos = pd.read_csv(db_location, sep=',')
# dffinal = limpiar_datos(datos)
# dffinal.shape
# dffinal
# # 3) Selección de numéricas y armado de matrices
# number_columns = dffinal.select_dtypes(include=[np.number]).columns
# test  = dffinal[number_columns].dropna()
# train = test.drop(columns=['precio_millones'])

# print(train.shape)  # esperado: (15912, 33)

# r_train_max = 0
# i_train= 0
# r_test_max = 0
# i_test = 0
# print("Buscando mejor random_state...", end='\r')
# for i in range (100):
#     print(f"=== Iteración {i+1} ===", end='\r')
#     X_train, X_test, y_train, y_test = train_test_split(
#         train, test["precio_millones"], test_size=0.3, random_state=i
#     )

#     reg = LinearRegression().fit(X_train, y_train)

#     r_train = reg.score(X_train, y_train)
#     r_test = reg.score(X_test, y_test)
#     if r_train > r_train_max:
#         r_train_max = r_train
#         i_train = i
#     if r_test > r_test_max:
#         r_test_max = r_test
#         i_test = i

# X_train, X_test, y_train, y_test = train_test_split(
#         train, test["precio_millones"], test_size=0.3, random_state=i_test
#     )
# print("_____________Máximo_____________")
# print("Train MAE:",  mean_absolute_error(y_train, reg.predict(X_train)))
# print("Test MAE:",   mean_absolute_error(y_test,  reg.predict(X_test)))
# print("Train RMSE:", np.sqrt(mean_squared_error(y_train, reg.predict(X_train))))
# print("Test RMSE:",  np.sqrt(mean_squared_error(y_test,  reg.predict(X_test))))
# print("Train R²:",   reg.score(X_train, y_train))
# print("Test R²:",    reg.score(X_test,  y_test))
# print("________________________________\n")



In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from typing import Optional


class FutAlpesRegressionPipeline(BaseEstimator, RegressorMixin):
    def __init__(self):
        print("Inicializando el pipeline de regresión FutAlpes...")
        self.pipeline = Pipeline([
            ('regressor', LinearRegression())
        ])

        self.model = LinearRegression()
        self.trained = False
        self.feature_columns_: Optional[list[str]] = None

    

    def fit(self, X: pd.DataFrame, y = None):
        print("Entrenando el modelo...")
        data = X.copy()
        
        dffinal = limpiar_datos(data)
        number_columns = dffinal.select_dtypes(include=[np.number]).columns
        data_test  = dffinal[number_columns].dropna()
        data_train = data_test.drop(columns=['precio_millones'])
        
        print(number_columns)
        r_train_max = 0
        i_train = 0
        r_test_max = 0
        i_test = 0

        self.feature_columns_ = data_train.columns.tolist()

        print("Buscando mejor random_state...", end='\r')
        for i in range(0):
            X_train, X_test, y_train, y_test = train_test_split(
            data_train, data_test['precio_millones'], test_size=0.3, random_state=i
            )
            self.pipeline.fit(X_train, y_train)
            r_train = self.pipeline.score(X_train, y_train)
            r_test = self.pipeline.score(X_test, y_test)
            if r_train > r_train_max:
                r_train_max = r_train
                i_train = i
            if r_test > r_test_max:
                r_test_max = r_test
                i_test = i

        X_train, X_test, y_train, y_test = train_test_split(
            data_train, data_test['precio_millones'], test_size=0.3, random_state=i_test
        )
        self.pipeline.fit(X_train, y_train)
        y_train_pred = self.pipeline.predict(X_train)
        y_test_pred = self.pipeline.predict(X_test)
        print("====== Model Performance ======")
        print("Train MAE:", mean_absolute_error(y_train, y_train_pred))
        print("Test MAE:", mean_absolute_error(y_test, y_test_pred))
        print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))
        print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))
        print("Train R²:", r2_score(y_train, y_train_pred))
        print("Test R²:", r2_score(y_test, y_test_pred))
        print("===============================\n")

        self.pipeline.fit(data_train, data_test["precio_millones"])
        self.is_fitted_ = True
        
        return self
        
    def arreglar_datos(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        df = limpiar_datos(df)
        return df
    
    def predict(self, X: pd.DataFrame) -> np.ndarray:
        if not self.is_fitted_:
            raise RuntimeError("El modelo no está entrenado. Llama primero a .fit().")

        df = limpiar_datos(X.copy())
        df_num = df.select_dtypes(include=[np.number]).copy()

        # Asegurar que existen todas las columnas usadas en entrenamiento
        # (si faltan, las creamos con 0; si sobran, las ignoramos)
        for col in self.feature_columns_:
            if col not in df_num.columns:
                df_num[col] = 0.0

        X_pred = df_num[self.feature_columns_].copy()

        # Cuidado: si llega a estar la columna objetivo en X, quítala
        if 'precio_millones' in X_pred.columns:
            X_pred = X_pred.drop(columns=['precio_millones'])

        # Eliminar filas con NaN para evitar errores
        X_pred = X_pred.dropna()

        return self.pipeline.predict(X_pred)

In [330]:
datos = pd.read_csv(db_location, sep=',')

modelo = FutAlpesRegressionPipeline()
modelo.fit(datos)

Inicializando el pipeline de regresión FutAlpes...
Entrenando el modelo...
Index(['Edad', 'Goles', 'Tiros Totales', 'xG', 'npxG', 'xAG',
       'Acciones_que_crean_tiros', 'Pases_intentados', 'Pases_progresivos',
       'Regates_exitosos',
       ...
       'Nacionalidad_tr TUR', 'Nacionalidad_ua UKR', 'Nacionalidad_us USA',
       'Nacionalidad_uy URU', 'Nacionalidad_uz UZB', 'Nacionalidad_ve VEN',
       'Nacionalidad_wls WAL', 'Nacionalidad_xk KVX', 'Nacionalidad_zm ZAM',
       'Nacionalidad_zw ZIM'],
      dtype='object', length=135)
====== Model Performance ======
Train MAE: 12724225.877899002
Test MAE: 13205673.33439101
Train RMSE: 19380710.524729468
Test RMSE: 20530717.289175015
Train R²: 0.27962170903158334
Test R²: 0.2595021507891736



FutAlpesRegressionPipeline()

# Guardar

In [331]:
datos = pd.read_csv(db_location, sep=',')
model_pipeline = FutAlpesRegressionPipeline()
model_pipeline.fit(datos)

dump(model_pipeline, "pipeline.joblib")
print("Modelo exportado como model_pipeline.joblib\n")

Inicializando el pipeline de regresión FutAlpes...
Entrenando el modelo...
Index(['Edad', 'Goles', 'Tiros Totales', 'xG', 'npxG', 'xAG',
       'Acciones_que_crean_tiros', 'Pases_intentados', 'Pases_progresivos',
       'Regates_exitosos',
       ...
       'Nacionalidad_tr TUR', 'Nacionalidad_ua UKR', 'Nacionalidad_us USA',
       'Nacionalidad_uy URU', 'Nacionalidad_uz UZB', 'Nacionalidad_ve VEN',
       'Nacionalidad_wls WAL', 'Nacionalidad_xk KVX', 'Nacionalidad_zm ZAM',
       'Nacionalidad_zw ZIM'],
      dtype='object', length=135)
====== Model Performance ======
Train MAE: 12724225.877899002
Test MAE: 13205673.33439101
Train RMSE: 19380710.524729468
Test RMSE: 20530717.289175015
Train R²: 0.27962170903158334
Test R²: 0.2595021507891736

Modelo exportado como model_pipeline.joblib



In [332]:
datos = pd.read_csv(db_location, sep=',')
model_pipeline = FutAlpesRegressionPipeline()
model_pipeline.fit(datos)

dump(model_pipeline, "pipeline.joblib")
print("Modelo exportado como model_pipeline.joblib\n")

Inicializando el pipeline de regresión FutAlpes...
Entrenando el modelo...
Index(['Edad', 'Goles', 'Tiros Totales', 'xG', 'npxG', 'xAG',
       'Acciones_que_crean_tiros', 'Pases_intentados', 'Pases_progresivos',
       'Regates_exitosos',
       ...
       'Nacionalidad_tr TUR', 'Nacionalidad_ua UKR', 'Nacionalidad_us USA',
       'Nacionalidad_uy URU', 'Nacionalidad_uz UZB', 'Nacionalidad_ve VEN',
       'Nacionalidad_wls WAL', 'Nacionalidad_xk KVX', 'Nacionalidad_zm ZAM',
       'Nacionalidad_zw ZIM'],
      dtype='object', length=135)
====== Model Performance ======
Train MAE: 12724225.877899002
Test MAE: 13205673.33439101
Train RMSE: 19380710.524729468
Test RMSE: 20530717.289175015
Train R²: 0.27962170903158334
Test R²: 0.2595021507891736

Modelo exportado como model_pipeline.joblib



In [333]:
pipeline = load("pipeline.joblib")

In [336]:
db_location = './Datos/datos_entrenamiento_laboratorio1(train_data).csv'
datos = pd.read_csv(db_location, sep=',')

pipeline.predict(datos.head())

C:\Users\diego\AppData\Local\Temp\ipykernel_24880\3481256170.py:91: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_num[col] = 0.0
C:\Users\diego\AppData\Local\Temp\ipykernel_24880\3481256170.py:91: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_num[col] = 0.0
C:\Users\diego\AppData\Local\Temp\ipykernel_24880\3481256170.py:91: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(ax

array([23877838.79123343, 44194858.70463109, 28222675.32660525])